In [2]:
import psycopg2 as ps
import pandas as pd
from sqlalchemy import create_engine
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler


In [3]:

conn = ps.connect(database='admission_predict_db',
                        user='postgres', password='88042425', 
                        host='127.0.0.1', port='5432'
)
  
# conn.autocommit = True
cursor = conn.cursor()
print("Connected!")

Connected!


In [4]:
sql3 = '''select * from Admission_Predict;'''
cursor.execute(sql3)
for i in cursor.fetchall():
    print(i)

('1', '337', '118', '4', '4.5', '4.5', '9.65', '1', '0.92')
('2', '324', '107', '4', '4', '4.5', '8.87', '1', '0.76')
('3', '316', '104', '3', '3', '3.5', '8', '1', '0.72')
('4', '322', '110', '3', '3.5', '2.5', '8.67', '1', '0.8')
('5', '314', '103', '2', '2', '3', '8.21', '0', '0.65')
('6', '330', '115', '5', '4.5', '3', '9.34', '1', '0.9')
('7', '321', '109', '3', '3', '4', '8.2', '1', '0.75')
('8', '308', '101', '2', '3', '4', '7.9', '0', '0.68')
('9', '302', '102', '1', '2', '1.5', '8', '0', '0.5')
('10', '323', '108', '3', '3.5', '3', '8.6', '0', '0.45')
('11', '325', '106', '3', '3.5', '4', '8.4', '1', '0.52')
('12', '327', '111', '4', '4', '4.5', '9', '1', '0.84')
('13', '328', '112', '4', '4', '4.5', '9.1', '1', '0.78')
('14', '307', '109', '3', '4', '3', '8', '1', '0.62')
('15', '311', '104', '3', '3.5', '2', '8.2', '1', '0.61')
('16', '314', '105', '3', '3.5', '2.5', '8.3', '0', '0.54')
('17', '317', '107', '3', '4', '3', '8.7', '0', '0.66')
('18', '319', '106', '3', '4', '3

In [5]:
df = pd.read_sql_query('''SELECT * FROM Admission_Predict''', con=conn)
df

C:\Users\16476\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4,4.5,8.87,1,0.76
2,3,316,104,3,3,3.5,8,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.8
4,5,314,103,2,2,3,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332,108,5,4.5,4,9.02,1,0.87
496,497,337,117,5,5,5,9.87,1,0.96
497,498,330,120,5,4.5,5,9.56,1,0.93
498,499,312,103,4,4,5,8.43,0,0.73


In [6]:
#converting the data from object to integer and float
df['Serial No.']=df['Serial No.'].astype(int)
df['GRE Score']=df['GRE Score'].astype(int)
df['TOEFL Score']=df['TOEFL Score'].astype(int)
df['University Rating']=df['University Rating'].astype(int)
df['SOP']=df['SOP'].astype(float)
df['LOR ']=df['LOR '].astype(float)
df['CGPA']=df['CGPA'].astype(float)
df['Research']=df['Research'].astype(int)
df['Chance of Admit ']=df['Chance of Admit '].astype(float)

In [7]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [8]:

X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.20,random_state=42)

# sc=StandardScaler()
# X_train=sc.fit_transform(X_train)
# X_test=sc.fit_transform(X_test)

classifier=linear_model.LinearRegression()
classifier.fit(X_train,y_train)

LinearRegression()

In [9]:
X_train.shape

(400, 8)

In [10]:
y_pred=classifier.predict(X_test)
y_pred

array([0.92671922, 0.77892806, 0.58584082, 0.70033329, 0.80448871,
       0.87586753, 0.48485782, 0.63648523, 0.80460912, 0.82275197,
       0.70344257, 0.72349461, 0.67134179, 0.92329979, 0.83506125,
       0.51923199, 0.86176246, 0.57346908, 0.53804446, 0.58000969,
       0.68193201, 0.5729244 , 0.72311442, 0.8036031 , 0.7663601 ,
       0.6204863 , 0.97205143, 0.84589389, 0.61484912, 0.75132286,
       0.57797234, 0.74074559, 0.56592273, 0.85345827, 0.63391827,
       0.74358402, 0.5654231 , 0.94281869, 0.6378805 , 0.69584915,
       0.98183147, 0.55758694, 0.68671005, 0.85348383, 0.92207356,
       0.56286619, 0.93783368, 0.81694811, 0.80861489, 0.90646329,
       0.87093318, 0.55607175, 0.6939115 , 0.53761435, 0.94962498,
       0.5826588 , 0.94844479, 0.72263978, 0.65694239, 0.48493317,
       0.64803091, 0.67586289, 0.60396804, 0.61893602, 0.42228723,
       0.57522282, 0.88508434, 0.87817478, 0.65395717, 0.71741528,
       0.62311373, 0.794649  , 0.69122223, 0.56807461, 0.55531

In [11]:
errors = mean_squared_error(y_test, y_pred)
print("mean square error:",errors)

mean square error: 0.003551180545098702


In [12]:
errors1 = mean_absolute_error(y_test, y_pred)
print("mean absolute error:",errors1)

mean absolute error: 0.04325885259545276


In [13]:
#unknown case 
classifier.predict([[800,300,100,1,3,3,8.5,1]])

array([0.69350184])